In [ ]:
import os
import libtts

import numpy as np

import matplotlib.pyplot as plt

## aux functions

In [ ]:
from plyfile import PlyData
import numpy as np

In [ ]:
def show_ply_point_summary(filepath):
    """
    Reads a PLY file and prints a summary of its point data.

    Args:
        filepath (str): The path to the PLY file.
    """
    try:
        plydata = PlyData.read(filepath)
        
        # show element names and properties
        print(f"Reading PLY file: {filepath}")
        print("Elements in the PLY file:")
        for element in plydata.elements:
            print(f"  - {element.name}: {len(element.data)} points")
            print(f"    Properties:")
            for prop in element.data.dtype.names:
                print(f"      - {prop}: {element.data.dtype[prop]}")
        print("\nPoint data summary:")

        element_names = [element.name for element in plydata.elements]
        if 'vertex' in element_names:
            vertex_data = plydata['vertex'].data
            print(f"Number of points: {len(vertex_data)}")

            # # Print information about the properties (e.g., x, y, z, color)
            # print("Point properties and their data types:")
            # for prop_name in vertex_data.dtype.names:
            #     print(f"  - {prop_name}: {vertex_data.dtype[prop_name]}")

            # Print min/max values for specific properties 
            # if # of pts is relatively small, otherwise it may take too long
            if len(vertex_data) < 1000000 and \
               'x' in vertex_data.dtype.names and \
               'y' in vertex_data.dtype.names and \
               'z' in vertex_data.dtype.names:
                print("\nCoordinate ranges:")
                print(f"  X-range: [{vertex_data['x'].min():.4f}, {vertex_data['x'].max():.4f}]")
                print(f"  Y-range: [{vertex_data['y'].min():.4f}, {vertex_data['y'].max():.4f}]")
                print(f"  Z-range: [{vertex_data['z'].min():.4f}, {vertex_data['z'].max():.4f}]")
        else:
            print(f"No 'vertex' element found in '{filepath}'.")
            print("Available elements:", [el.name for el in plydata.elements])

    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'")
    except Exception as e:
        print(f"An error occurred: {e}")

## ground detection

In [ ]:
# set input file
infile = f"tree_228.pts"

In [ ]:
# set output files
out_gd_file = f"tree_228_gd_xyzh.pts"
out_veg_file = f"tree_228_veg_xyzh.pts"
out_gd_file, out_veg_file = libtts.run_ground_detection(infile = infile, out_gd_file = out_gd_file, out_veg_file = out_veg_file, 
                                                        grid_size=0.1, height_threshold = 0.5)


In [ ]:
inpts = np.loadtxt(infile)
inpts.shape

In [ ]:
# detect ground points
ground_grid_pts, grid_x, grid_y, grid_z = libtts.detect_ground(inpts, grid_size=0.1, outlier_std_dev=1.0, gaussian_kernel_size=5)
libtts.plot_ground_model(grid_x, grid_y, grid_z)

In [ ]:
# calculate height above ground
# and separate vegetation points from ground points
pts_xyzh = libtts.calculate_height_above_ground(inpts, grid_x, grid_y, grid_z)
heights = pts_xyzh[:, 3]
height_threshold = 0.5
veg_mask = heights > height_threshold

ground_points = inpts[~veg_mask]
veg_points = inpts[veg_mask]

print(f"Ground points: {ground_points.shape[0]}, Vegetation points: {veg_points.shape[0]}")
# save
gd_file =  infile.replace(".pts", "_gd_xyzh.pts")
vg_file =  infile.replace(".pts", "_vg_xyzh.pts")

np.savetxt(gd_file, ground_points, fmt="%.3f")
np.savetxt(vg_file, veg_points, fmt="%.3f")

In [ ]:
# or, we can use the classify_ground_and_vegetation function
# which does the same thing in one step
gd_file =  infile.replace(".pts", "_gd_xyzh.pts")
vg_file =  infile.replace(".pts", "_vg_xyzh.pts")
gd_pts, vg_pts = libtts.classify_ground_and_vegetation(inpts, height_threshold=0.5, out_gd_file=gd_file, out_veg_file=vg_file,
                                                       grid_size=0.1, outlier_std_dev=1.0, gaussian_kernel_size=5)

## tree detection and extraction

In [ ]:
tls_veg_file = "close_stems_3.pts"

In [ ]:
pts = np.loadtxt(tls_veg_file)
pts.shape

In [ ]:
# should be xyzh, if h is not present, it will be added as z
if pts.shape[1] == 3:
    pts = np.column_stack((pts, pts[:, 2]))
pts.shape

In [ ]:
tree_bases = libtts.filter_tree_bases(pts, height_min=0.5, height_max=1.0, knn=4, max_avg_dist=1.0)
tree_bases.shape

In [ ]:
# plot tree bases points
plt.figure(figsize=(5, 5))
plt.scatter(tree_bases[:, 0], tree_bases[:, 1], c=tree_bases[:, 2], s=2, cmap='viridis')
plt.colorbar(label='ID')
plt.title('Tree Bases Points')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')
plt.grid()
plt.show()
plt.close()

In [ ]:
labeled_trees = libtts.cluster_points_dbscan(tree_bases, eps=0.2, min_samples=5, use_2d=True)
labeled_trees.shape # x y z knn-distance label

In [ ]:
# plot labeled trees
plt.figure(figsize=(5, 5))
plt.scatter(labeled_trees[:, 0], labeled_trees[:, 1], c=labeled_trees[:, -1], s=2, cmap='Set1')
plt.colorbar(label='Cluster ID')
plt.title('Labeled Trees')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')
plt.grid()
plt.show()
plt.close()

In [ ]:
# or, we can use the detect_trees function
# which does the same thing in one step
if pts.shape[1] == 3:
    pts = np.column_stack((pts, pts[:, 2]))
tree_locs = libtts.detect_trees(pts, height_min=0.5, height_max=1.0, knn=4, max_avg_dist=1.0, eps=0.2, min_samples=5, use_2d=True)
tree_locs.shape

In [ ]:
# plot labeled trees
plt.figure(figsize=(5, 5))
plt.scatter(tree_locs[:, 0], tree_locs[:, 1], c=tree_locs[:, -1], s=2, cmap='Set1') # 
plt.colorbar(label='Cluster ID')
plt.title('Labeled Trees')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')
plt.grid()
plt.show()
plt.close()

In [ ]:
tree_locfile = libtts.run_tree_detection(infile = tls_veg_file, outfile= "tree_locations.pts", 
                                         height_min=0.5, height_max=1.0, max_dist=1.0,
                                         eps=0.2, min_samples=5)

## tree extraction

In [ ]:
### generate alpha shape
th_alpha_sq = 0.01
as_file = libtts.generate_alpha_shape(tls_veg_file, th_alpha_sq)
print("Alpha shape file:", as_file)

In [ ]:
### tree segmentation
as_file = f"{tls_veg_file[:-4]}_a0.010.off"
treeloc_file = f"{tls_veg_file[:-4]}_treeloc_xyzl.pts"
print(f"{as_file=}, {treeloc_file=}")

segfile = libtts.tls_extract_single_trees(as_file, treeloc_file, th_p2trunk_distance=0.2, th_search_radius=0.25)

print(segfile) 

In [ ]:
### label remaining points
tls_veg_file = "close_stems_3.pts"
segfile = f"{tls_veg_file[:-4]}_a0.010_lbl.pts"

all_pts = np.loadtxt(tls_veg_file)
initial_labels = np.loadtxt(segfile)
all_lbl_file = f"{tls_veg_file[:-4]}_lbl_all.pts"
all_pts_lbls = libtts.label_points_region_growing(all_pts, initial_labels, search_radius=0.5, out_file=all_lbl_file)

## downsampling

In [ ]:
infile = "t109_roi.pts"

In [ ]:
### make an alpha shape
th_alpha_sq = 0.01
as_file = libtts.generate_alpha_shape(infile, th_alpha_sq)

In [ ]:
as_file = "t109_roi_a0.010.off"

In [ ]:
libtts.downsample_points_from_mesh(as_file, th_avg_dis=0.1)

In [ ]:
infile = "t109_roi.pts"
dsfile = libtts.run_downsampling(infile, input_data = "pts", th_alpha_sq=0.01, th_avg_dis=0.1)


In [ ]:
as_file = "t109_roi_a0.010.off"
dsfile = libtts.run_downsampling(as_file, input_data = "mesh", th_alpha_sq=0.01, th_avg_dis=0.1)

In [ ]:
overseg_file = "t109_roi_a0.010_lbl.pts"
dsfile = libtts.run_downsampling(overseg_file, input_data = "overseg", th_alpha_sq=0.01, th_avg_dis=0.1)